In [84]:
import csv
import os
import re

from dataclasses import dataclass, field

@dataclass
class Lesson:
    subj: str = None
    module_num: int = 0
    lesson_num: int = 0
    url: str = None
    student_docx: str = None
    student_pdf: str = None
    teacher_docx: str = None
    teacher_pdf: str = None
    status: str = None
    notes: str = ""
    keywords: set = field(default_factory=lambda: set())

modules = []
lessons = []
subjects = set()
lsn = None
with open('../data/EngageNY/content.tsv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            a = row[0].strip()
            b = row[1].strip()
            c = row[2].strip()
            d = row[3].strip()
            g = row[6].strip()
            h = row[7].strip()
            i = row[8].strip()
            if len(a)>0:
                  modules.append(a)
            if len(b)>0:
                  if lsn is not None:
                      lessons.append(lsn)
                      if len(lsn.keywords) > 0:
                          print(lsn)
                  lsn = Lesson(url=b)
                  # print(f'\t{b}')
            if len(d)>0:
                  if lsn.subj is None:
                      # print(f"\t<<{d}>>")
                      subj, _ = d.split(' Module ')
                      subj = subj.replace(' Mathematics', '').strip()
                      if subj=='Precalculus and Advanced Topics':
                          subj = 'Precalculus'
                      subjects.add(subj)
                      lsn.subj = subj
                      mod_id = re.match(r".*Module ([0-9]+).*",d).group(1)
                      lsn.module_num = int(mod_id)
                      les_id = re.match(r".* Lesson ([0-9]+).*",d).group(1)
                      # print (les_id)
                      lsn.lesson_num = int (les_id)
                      # print(lsn.subj,'--',lsn.module_num,'--',lsn.lesson_num)
            if len(c)>0:
                  is_pdf = bool (re.search(r"\.pdf", c))
                  is_docx = bool (re.search(r"\.docx", c))
                  is_teacher = bool (re.search(r"-teacher\.", c))
                  if is_teacher:
                      if is_pdf:
                          lsn.teacher_pdf = c
                      if is_docx:
                          lsn.teacher_docx = c
                  else:
                      if is_pdf:
                          lsn.student_pdf = c
                      if is_docx:
                          lsn.student_docx = c
            if len(g)>0:
                  lsn.status = g.lower()
            if len(h)>0:
                  if (len(lsn.notes)>0): 
                      lsn.notes += " "
                  lsn.notes += h
            if len(i)>0:
                  lsn.keywords = lsn.keywords.union( set(re.split('[,;] ', i)) )             
                  
            line_count += 1
    if lsn is not None:
        lessons.append(lsn)
    print(f'Processed {line_count} lines, {len(modules)} modules, {len(lessons)} lessons.') 
    
    subjects
    

Column names are Module:, Lesson:, Files:, , , , Reviewed, status:, Notes:, Keywords:
Lesson(subj='Grade 6', module_num=2, lesson_num=7, url='https://www.engageny.org/resource/grade-6-mathematics-module-2-topic-lesson-7', student_docx='https://www.engageny.org/file/40456/download/math-g6-m2-topic-a-lesson-7-student.docx?token=FWHtUFlM', student_pdf='https://www.engageny.org/file/40446/download/math-g6-m2-topic-a-lesson-7-student.pdf?token=G8t55kZI', teacher_docx='https://www.engageny.org/file/40461/download/math-g6-m2-topic-a-lesson-7-teacher.docx?token=l-iyq__o', teacher_pdf='https://www.engageny.org/file/40451/download/math-g6-m2-topic-a-lesson-7-teacher.pdf?token=YWRx9qQO', status='yes', notes='', keywords={'partitive interpretation', ' measurement interpretation', 'reciprocals'})
Lesson(subj='Grade 6', module_num=2, lesson_num=10, url='https://www.engageny.org/resource/grade-6-mathematics-module-2-topic-b-lesson-10', student_docx='https://www.engageny.org/file/40531/download/math-g

{'Algebra I',
 'Algebra II',
 'Geometry',
 'Grade 1',
 'Grade 2',
 'Grade 3',
 'Grade 4',
 'Grade 5',
 'Grade 6',
 'Grade 7',
 'Grade 8',
 'Kindergarten',
 'Precalculus'}

In [87]:
import modules.engageny as engageny
lessons, subjects = engageny.process_spreadsheet()

subjects

Processed 7012 lines, 1761 lessons.


{'Algebra I',
 'Algebra II',
 'Geometry',
 'Grade 1',
 'Grade 2',
 'Grade 3',
 'Grade 4',
 'Grade 5',
 'Grade 6',
 'Grade 7',
 'Grade 8',
 'Kindergarten',
 'Precalculus'}

In [103]:
from bs4 import BeautifulSoup as Bsoup
import requests
import re

std_refs = {}

def scan_engage_lesson(url):
    r  = requests.get(url)
    soup = Bsoup(r.text, 'html.parser')
    desc = None
    stds = set()
    title = soup.title.text
    div_w_sum = soup.find_all('div', {"class": "field-type-text-with-summary"})
    for d in div_w_sum:
        desc = d.text.strip()
        break
        
    std_ref = re.compile("^([A -Z0-9a-d]+\.){2,5}[A-Z0-9a-d]+$")        
    links = soup.find_all('a')
    for l in links:
        t = l.text.strip()
        if std_ref.search(t):
            stds.add(t);
            std_refs[t] = f'https://www.engageny.org{l["href"]}'
    return title,desc,stds
                  
                  
for i,les in enumerate(lessons):
    if i%11 !=7 :
        continue;

    title,desc,stds = engageny.scan_engage_lesson(les.url)
    print(les.url)
    print(title)
    print(desc)
    print(', '.join(stds))            
                         
    print('= = = = = =')
    continue
    


https://www.engageny.org/resource/kindergarten-mathematics-module-1-topic-c-lesson-8
Kindergarten Mathematics Module 1, Topic C, Lesson 8 | EngageNY
Objective: Answer how many questions to 5 in linear configurations (5-group) with 4 in an array configuration. Compare ways to count 5 fingers.
K.CC.6
= = = = = =
https://www.engageny.org/resource/kindergarten-mathematics-module-1-topic-e-lesson-19
Kindergarten Mathematics Module 1, Topic E, Lesson 19 | EngageNY
Objective: Count 5─7 linking cubes in linear configurations. Match with numeral 7. Count on fingers from 1 to 7 and connect to 5-group images.
K.CC.4.a, K.CC.3, K.CC.5, K.CC.4.b
= = = = = =
https://www.engageny.org/resource/kindergarten-mathematics-module-1-topic-g-lesson-30
Kindergarten Mathematics Module 1, Topic G, Lesson 30 | EngageNY
Objective: Exploration: Make math stairs from 1 to 10 in cooperative groups.
K.CC.4.a, K.CC.5, K.CC.4.b, K.CC.2, K.CC.4.c
= = = = = =
https://www.engageny.org/resource/kindergarten-mathematics-mod

KeyboardInterrupt: 

In [104]:
i = 0
for s in engageny.std_refs:
    i+=1
    if i%7!=3:
        continue;
    url = engageny.std_refs[s]
    print(s,url)
    desc, v, np = engageny.scan_std(url)
    # print(desc)
    print("Noun phrases:\n\t-", '\n\t- '.join(np))
    print("Verbs:\n\t-", '\n\t- '.join(v))



K.CC.4.a https://www.engageny.org/ccls-math/kcc4a
Noun phrases:
	- objects
	- the number names
	- the standard order
	- each object
	- only one number name
	- one and only one object
Verbs:
	- count
	- say
	- pair
K.MD.2 https://www.engageny.org/ccls-math/kmd2
Noun phrases:
	- two objects
	- a measurable attribute
	- which object
	- “more of”/“less
	- the attribute
	- the difference
	- example
	- the heights
	- two children
	- one child
	- taller/shorter
Verbs:
	- compare
	- see
	- have
	- describe
	- compare
	- describe
